### Import Libraries

In [2]:
import pandas as pd
import sqlalchemy, os
from binance import BinanceSocketManager
from binance.client import Client
from dotenv import load_dotenv
load_dotenv()

True

### Environment variables

In [3]:
API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")

# Binance Client
client = Client(API_KEY, API_SECRET)

# SQLite EngineSQLite Engine
# engine = sqlalchemy.create_engine('sqlite:///SLPUSDTstream.db')

### Getting symbols (Level 1)

In [4]:
info = client.get_exchange_info()['symbols']

#This'll contain all symbols in Binance that has USDT
symbols_USDT = ""

for item in info:
    symbol = item['symbol']
    
    if 'USDT' in symbol:
        symbols_USDT+=symbol+','
        
symbols_USDT = symbols_USDT[:-1]

with open('pairs.txt','w') as file:
    file.write(symbols_USDT)

### Filter - percent change in 24hrs

In [8]:
symbols = symbols_USDT.split(',')

info = client.get_ticker()
# All symbols paired with corresponding 24hrs change percentile
all_change_pair = {}
# All symbols with USDT paired with corresponding 24hrs change percentile
USDT_change_pair = {}
# List of symbols with more than x% change within 24hrs
level1=[]

for item in info:
    all_change_pair[item['symbol']] = float(item['priceChangePercent'])

for item in all_change_pair:
    if item in symbols:
        USDT_change_pair[item] = all_change_pair[item]

# Sort by 24hrs change percentage
sorted_USDT_change_pair = {pair[0] : pair[1] for pair in sorted(USDT_change_pair.items(), key=lambda x: x[1], reverse=True)}

for x in sorted_USDT_change_pair.items():
    if x[1] > 10.0:
        level1.append(x[0])

with open('pairs_level1.txt','w') as file:
    list_level1=''
    for item in level1:
        list_level1+=item+','
    file.write(list_level1[:-1])

# level1 contains the list of symbols that pass the first criteria

### Filter - percent change in 6hrs

In [9]:
# Method for getting price
def get_price(ls):
    avg = (float(ls[1])+float(ls[4]))/2
    price = float("{:.4f}".format(avg))
    return price

# Method for getting percent change in 6 hour
def get_percent_change(ls_1,ls_2):
    price_1 = get_price(ls_1)
    price_2 = get_price(ls_2)
    
    change = (price_2 - price_1)/price_1
    
    change_in_percent = float("{:.4f}".format(change*100))
    
    return change_in_percent

def filter2(list_of_symbols):
    from datetime import datetime, timedelta
    
    level2 = []
    
    for symbol in list_of_symbols:
        
        klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1MINUTE,str(datetime.now() - timedelta(hours=12)), str(datetime.now()))
        
        percent_change = get_percent_change(klines[0],klines[-1])
        
        if percent_change > 5:
            level2.append(symbol)
            print(symbol,percent_change)
            
    return level2

In [10]:
level2 = filter2(level1)
with open('pairs_level2.txt','w') as file:
    list_level2=''
    for item in level2:
        list_level2+=item+','
    file.write(list_level2[:-1])

LINAUSDT 11.57
RUNEUSDT 5.18
PYRUSDT 7.95
SANDUSDT 8.04
UTKUSDT 5.43
GLMRUSDT 5.63


### Function for creating dataframe

In [7]:
def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:,['s','E','p']]
    df.columns = ['symbol','Time','Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time , unit='ms')
    return df

In [ ]:
while True:
    await socket.__aenter__()
    msg = await socket.recv()
    while msg['e']=='error':
        msg = await socket.recv()
    frame = createframe(msg)
    frame.to_sql('SLPUSDT', engine, if_exists='append', index=False)
    print(frame)

    symbol                    Time   Price
0  SLPUSDT 2022-02-19 18:31:43.764  0.0238
    symbol                    Time   Price
0  SLPUSDT 2022-02-19 18:31:43.764  0.0238
    symbol                    Time   Price
0  SLPUSDT 2022-02-19 18:31:43.764  0.0238
    symbol                    Time   Price
0  SLPUSDT 2022-02-19 18:31:43.764  0.0238
    symbol                    Time   Price
0  SLPUSDT 2022-02-19 18:31:43.764  0.0238
    symbol                    Time   Price
0  SLPUSDT 2022-02-19 18:31:43.764  0.0238
    symbol                    Time   Price
0  SLPUSDT 2022-02-19 18:31:47.173  0.0238
    symbol                    Time   Price
0  SLPUSDT 2022-02-19 18:31:47.173  0.0238
    symbol                    Time   Price
0  SLPUSDT 2022-02-19 18:31:48.330  0.0239
    symbol                    Time   Price
0  SLPUSDT 2022-02-19 18:31:48.330  0.0239
    symbol                    Time   Price
0  SLPUSDT 2022-02-19 18:31:48.761  0.0238
    symbol                    Time   Price
0  SLPUSDT 

In [ ]:
msg=await socket.recv()
if msg['e']:
    print (msg['e'])
print(msg)

error
{'e': 'error', 'm': 'Queue overflow. Message not filled'}
